In [1]:
import pandas as pd
import pandas_datareader.data as web
import datetime as dt
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import seaborn as sns
# NLTK VADER for sentiment analysis
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')
finwiz_url = 'https://finviz.com/quote.ashx?t='


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/nikatsaouni/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


/Users/nikatsaouni/anaconda3/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [1]:
import pandas as pd
import pandas_datareader.data as web

In [3]:
df = web.DataReader('WIKI/AAPL', 'quandl', '2015-01-01', '2015-01-05')

ValueError: The Quandl API key must be provided either through the api_key variable or through the environmental variable QUANDL_API_KEY.

In [350]:
### Read news
def read_news_data(news_after, path_to_news ):
    news = pd.read_csv(path_to_news)
    ## Keep news after '2016-01-01'
    news = news[news.release_date > news_after] 
    news = news.reset_index()
    return news

def sentiment_analysis(news):
    ##### Sentiment analysis 

    # Instantiate the sentiment intensity analyzer
    vader = SentimentIntensityAnalyzer()
    # Iterate through the headlines and get the polarity scores using vader
    scores = news['title'].apply(vader.polarity_scores).tolist()
    # Convert the 'scores' list of dicts into a DataFrame
    scores_df = pd.DataFrame(scores)
    # Join the DataFrames of the news and the list of dicts
    df_usq = news.join(scores_df, rsuffix='_right')
    #print(reuters_aapl.isna().sum())
    # Convert the date column from string to datetime
    df_usq['release_date'] = pd.to_datetime(df_usq.release_date).dt.date
    df_usq = df_usq.reset_index()
    return df_usq, scores_df

def group_data(sent_anal_df,companies_names):
    ### get names of the unique companies
    news_with_sent_anal = pd.DataFrame()
    ### Group each company according to the data
    for i in range(len(companies_names)):
        reuters_test = pd.DataFrame()
        sent_anal_df_new = sent_anal_df[sent_anal_df.ticker==companies_names[i]]
        reuters_test = sent_anal_df_new.groupby(['ticker','release_date']).mean() 
        reuters_test['news_amount'] = sent_anal_df_new.groupby(['ticker','release_date']).count()['index']
        reuters_test['comp_aggr'] = reuters_test['news_amount']*reuters_test['compound']    
        news_with_sent_anal = news_with_sent_anal.append(reuters_test)

    return news_with_sent_anal

def get_stock_prices_yahoo(companies_names):
    stock_pr = pd.DataFrame()
    index_of_errors = []
    for i in range(len(companies_names)):
        try:
            df = web.DataReader(companies_names[i],'yahoo',news_with_sent_anal.loc[companies_names[i]].index.min(),news_with_sent_anal.loc[companies_names[i]].index.max())
            df['ticker'] = companies_names[i]
            stock_pr = stock_pr.append(df)
        except:
            index_of_errors.append(i)
            pass 
    return stock_pr, index_of_errors

def mean_func(numbers):
    return float(sum(numbers)) / max(len(numbers), 1)


### add stock prices in the news dataframe for 2 days, 1 week, 1 month
def add_trend_columns():
    d1 = pd.DataFrame()
    d2 = pd.DataFrame()
    d3 = pd.DataFrame()
    for i in range(len(companies_names)):
        df = stock_pr.loc[stock_pr['ticker'] == companies_names[i]]
        for j in range(df.shape[0]):          
            try:               
                d1 = d1.append(pd.Series(mean_func(df.iloc[j:j+2]['Adj Close'])),ignore_index=True,sort=False)
            except:
                d1 = d1.append(df.iloc[j]['Adj Close'])
            pass        
            try:
                d2 = d2.append(pd.Series(mean_func(df.iloc[j:j+8]['Adj Close'])),ignore_index=True,sort=False)
            except:
                ## how many days left
                days_left = df.shape[0] - j               
                d2 = d2.append(pd.Series(mean_func(df.iloc[j:j+days_left]['Adj Close'])),ignore_index=True,sort=False)
            pass
            try:
                d3 = d3.append(pd.Series(mean_func(df.iloc[j:j+31]['Adj Close'])),ignore_index=True,sort=False)
            except:
                ## how many days left
                days_left = df.shape[0] - j               
                d3 = d3.append(pd.Series(mean_func(df.iloc[j:j+days_left]['Adj Close'])),ignore_index=True,sort=False)
            pass
    d1.columns = ['Next_day']
    d2.columns = ['Next_week']
    d3.columns = ['Next_month'] 
    return d1, d2, d3


In [166]:
news_after = '2016-01-01'
path_to_news = '/Users/nikatsaouni/Documents/NewsAnalyzer/us_equities_news_dataset.csv'
#  Read news from .csv file 
news = read_news_data(news_after,path_to_news)


KeyboardInterrupt: 

In [4]:
#  SENTIMENTAL ANALYSIS on the news
sent_anal_df, scores_df = sentiment_analysis(news)


In [5]:
# Group dataset for each company per day 
companies_names = sent_anal_df.ticker.unique()
news_with_sent_anal = group_data(sent_anal_df,companies_names)
news_with_sent_anal


level_0          index             id  \
ticker release_date                                                
NIO    2018-09-12        62.000000      62.000000  221577.000000   
       2018-09-14        61.000000      61.000000  221576.000000   
       2018-09-24        60.000000      60.000000  221575.000000   
       2018-10-03        54.000000      54.000000  221569.000000   
       2018-10-09        75.000000      75.000000  221590.000000   
...                            ...            ...            ...   
AMD    2020-02-05    179242.000000  220352.000000  441868.000000   
       2020-02-06    179197.500000  220307.500000  441823.500000   
       2020-02-11    179259.333333  220369.333333  441885.333333   
       2020-02-12    179291.000000  220401.000000  441917.000000   
       2020-02-13    179292.000000  220402.000000  441918.000000   

                       article_id     neg       neu       pos  compound  \
ticker release_date                                                       
NIO    2018-09-12    1.606489e+06  0.0000  1.000000  0.000000    0.0000   
       2018-09-14    1.610808e+06  0.0000  1.000000  0.000000    0.0000   
       2018-09-24    1.621487e+06  0.0000  1.000000  0.000000    0.0000   
       2018-10-03    1.632123e+06  0.0000  1.000000  0.000000    0.0000   
       2018-10-09    1.637643e+06  0.0000  1.000000  0.000000    0.0000   
...                           ...     ...       ...       ...       ...   
AMD    2020-02-05    1.012906e+08  0.0000  0.907500  0.092500    0.1806   
       2020-02-06    2.077960e+06  0.1755  0.665500  0.159000    0.1175   
       2020-02-11    1.343647e+08  0.0000  0.840333  0.159667    0.1858   
       2020-02-12    2.005071e+08  0.0000  0.833000  0.167000    0.2023   
       2020-02-13    2.005073e+08  0.3360  0.479000  0.185000   -0.2960   

                     news_amount  comp_aggr  
ticker release_date                          
NIO    2018-09-12              1     0.0000  
       2018-09-14              1     0.0000  
       2018-09-24              1     0.0000  
       2018-10-03              1     0.0000  
       2018-10-09              1     0.0000  
...                          ...        ...  
AMD    2020-02-05              2     0.3612  
       2020-02-06              2     0.2350  
       2020-02-11              3     0.5574  
       2020-02-12              1     0.2023  
       2020-02-13              1    -0.2960  

[96378 rows x 10 columns]

In [6]:
### Retrieve the stock prices from yahoo
companies_names = sent_anal_df.ticker.unique()
stock_pr, index_of_errors = get_stock_prices_yahoo(companies_names)
stock_pr


,High,Low,Open,Close,Volume,Adj Close,ticker
Date,,,,,,,
2018-09-12,6.930000,5.350000,6.000000,6.600000,66849000.0,6.600000,NIO
2018-09-13,12.690000,6.520000,6.620000,11.600000,158346500.0,11.600000,NIO
2018-09-14,13.800000,9.220000,12.660000,9.900000,172473600.0,9.900000,NIO
2018-09-17,9.750000,8.500000,9.610000,8.500000,56323900.0,8.500000,NIO
2018-09-18,9.100000,7.670000,8.730000,7.680000,41827600.0,7.680000,NIO
...,...,...,...,...,...,...,...
2020-02-07,50.669998,48.639999,48.910000,49.730000,53107600.0,49.730000,AMD
2020-02-10,52.299999,49.349998,49.470001,52.259998,60971700.0,52.259998,AMD
2020-02-11,54.480000,52.910000,53.220001,53.799999,74719100.0,53.799999,AMD


In [19]:
### Delete news from companies that do not exist in the available stock prices
news_with_sent_anal = news_with_sent_anal.drop(companies_names[index_of_errors], level='ticker')
news_with_sent_anal


level_0          index             id  \
ticker release_date                                                
NIO    2018-09-12        62.000000      62.000000  221577.000000   
       2018-09-14        61.000000      61.000000  221576.000000   
       2018-09-24        60.000000      60.000000  221575.000000   
       2018-10-03        54.000000      54.000000  221569.000000   
       2018-10-09        75.000000      75.000000  221590.000000   
...                            ...            ...            ...   
AMD    2020-02-05    179242.000000  220352.000000  441868.000000   
       2020-02-06    179197.500000  220307.500000  441823.500000   
       2020-02-11    179259.333333  220369.333333  441885.333333   
       2020-02-12    179291.000000  220401.000000  441917.000000   
       2020-02-13    179292.000000  220402.000000  441918.000000   

                       article_id     neg       neu       pos  compound  \
ticker release_date                                                       
NIO    2018-09-12    1.606489e+06  0.0000  1.000000  0.000000    0.0000   
       2018-09-14    1.610808e+06  0.0000  1.000000  0.000000    0.0000   
       2018-09-24    1.621487e+06  0.0000  1.000000  0.000000    0.0000   
       2018-10-03    1.632123e+06  0.0000  1.000000  0.000000    0.0000   
       2018-10-09    1.637643e+06  0.0000  1.000000  0.000000    0.0000   
...                           ...     ...       ...       ...       ...   
AMD    2020-02-05    1.012906e+08  0.0000  0.907500  0.092500    0.1806   
       2020-02-06    2.077960e+06  0.1755  0.665500  0.159000    0.1175   
       2020-02-11    1.343647e+08  0.0000  0.840333  0.159667    0.1858   
       2020-02-12    2.005071e+08  0.0000  0.833000  0.167000    0.2023   
       2020-02-13    2.005073e+08  0.3360  0.479000  0.185000   -0.2960   

                     news_amount  comp_aggr  
ticker release_date                          
NIO    2018-09-12              1     0.0000  
       2018-09-14              1     0.0000  
       2018-09-24              1     0.0000  
       2018-10-03              1     0.0000  
       2018-10-09              1     0.0000  
...                          ...        ...  
AMD    2020-02-05              2     0.3612  
       2020-02-06              2     0.2350  
       2020-02-11              3     0.5574  
       2020-02-12              1     0.2023  
       2020-02-13              1    -0.2960  

[93706 rows x 10 columns]

In [351]:
### add stockprices in the news dataframe for 2 days, 1 week, 1 month
day, week, month = add_trend_columns()
    

In [352]:
stock_pr_reindex = stock_pr.reset_index()
prices_with_trends = pd.concat([stock_pr_reindex, day, week, month], axis=1, sort=False)
prices_with_trends


,Date,High,Low,Open,Close,Volume,Adj Close,ticker,Next_day,Next_week,Next_month
0,2018-09-12,6.930000,5.350000,6.000000,6.600000,66849000.0,6.600000,NIO,9.100000,8.768750,7.506774
1,2018-09-13,12.690000,6.520000,6.620000,11.600000,158346500.0,11.600000,NIO,10.750000,8.927500,7.501935
2,2018-09-14,13.800000,9.220000,12.660000,9.900000,172473600.0,9.900000,NIO,9.200000,8.425000,7.332581
3,2018-09-17,9.750000,8.500000,9.610000,8.500000,56323900.0,8.500000,NIO,8.090000,8.125000,7.212903
4,2018-09-18,9.100000,7.670000,8.730000,7.680000,41827600.0,7.680000,NIO,8.090000,7.938750,7.130645
...,...,...,...,...,...,...,...,...,...,...,...
659405,2020-02-07,50.669998,48.639999,48.910000,49.730000,53107600.0,49.730000,AMD,50.994999,52.841999,52.841999
659406,2020-02-10,52.299999,49.349998,49.470001,52.259998,60971700.0,52.259998,AMD,53.029999,53.619999,53.619999
659407,2020-02-11,54.480000,52.910000,53.220001,53.799999,74719100.0,53.799999,AMD,53.844999,54.073332,54.073332
659408,2020-02-12,54.849998,53.529999,54.529999,53.889999,45994900.0,53.889999,AMD,54.209999,54.209999,54.209999


In [357]:
prices_with_trends.to_csv('Prices_with_Trends.csv')  # where to save it, usually as a .pkl
news_with_sent_anal.to_csv('News.csv')